# Inverse Kinematics in Newton

[![ Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-35QaaoiXx6VDmVNBOEtmpLs9VBs)

This notebook demonstrates inverse kinematics (IK) in Newton using the Franka FR3 robot arm. We'll make the end-effector trace a square path in 3D space.

Key topics:
- Setting up IK objectives (position and rotation)
- Using the IK solver with Levenberg-Marquardt optimization
- Visualizing target positions and trajectory traces
- Real-time IK solving at 60 Hz


## Setup and Imports


In [ ]:
import newton
import newton.ik as ik
import newton.utils
import warp as wp
import numpy as np
from tqdm.notebook import trange


## What is Inverse Kinematics?

**Forward Kinematics (FK)**: Given joint angles → compute end-effector position

**Inverse Kinematics (IK)**: Given desired end-effector position → compute joint angles

IK is essential for:
- Task-space control (move hand to target)
- Motion planning (reach and grasp)
- Teleoperation and imitation learning

Newton's IK solver uses **Levenberg-Marquardt** optimization to minimize the error between current and target poses.


## Load Franka Robot

We'll use the Franka FR3 robot arm with a fixed base.


In [ ]:
# Build the robot model
builder = newton.ModelBuilder()
builder.add_urdf(
    newton.utils.download_asset("franka_emika_panda") / "urdf/fr3_franka_hand.urdf",
    floating=False,
)
builder.add_ground_plane()

# Finalize model
model = builder.finalize()

print(f"Model loaded: {model.body_count} bodies, {model.joint_count} joints")
print(f"Joint coordinates: {model.joint_coord_count}")


## Initialize State and End-Effector

Compute forward kinematics to get the initial end-effector pose.


In [ ]:
# Set some reasonable initial joint coordinates
model.joint_q.assign([-0.03275817, -0.46363345, -0.02417757, -2.2807403,  -0.01114806,  1.8172095, -0.05166471, 0., 0.])

# Create state and evaluate FK
state = model.state()
newton.eval_fk(model, model.joint_q, model.joint_qd, state)

# End-effector body index (hand TCP)
ee_index = 10

# Get initial end-effector transform
body_q_np = state.body_q.numpy()
ee_initial = wp.transform(*body_q_np[ee_index])
ee_pos = wp.transform_get_translation(ee_initial)
ee_rot = wp.transform_get_rotation(ee_initial)

print(f"End-effector index: {ee_index}")
print(f"Initial position: ({ee_pos[0]:.3f}, {ee_pos[1]:.3f}, {ee_pos[2]:.3f})")
print(f"Initial rotation: ({ee_rot[0]:.3f}, {ee_rot[1]:.3f}, {ee_rot[2]:.3f}, {ee_rot[3]:.3f})")

viewer = newton.viewer.ViewerRerun(keep_historical_data=True)
viewer.set_model(model)
viewer.log_state(state)
viewer


## Define Square Trajectory

Create a square path in the XZ plane for the end-effector to follow.


In [ ]:
# Square parameters
square_size = 0.2  # 20 cm sides
center_x = ee_pos[0]
center_y = ee_pos[1]
center_z = ee_pos[2]

# Generate square waypoints (4 corners + return to start)
num_points = 200
points_per_side = num_points // 4

# Create square in XZ plane
square_path = []

# Bottom edge (left to right)
for i in range(points_per_side):
    t = i / points_per_side
    x = center_x - square_size/2 + t * square_size
    y = center_y
    z = center_z - square_size/2
    square_path.append([x, y, z])

# Right edge (bottom to top)
for i in range(points_per_side):
    t = i / points_per_side
    x = center_x + square_size/2
    y = center_y
    z = center_z - square_size/2 + t * square_size
    square_path.append([x, y, z])

# Top edge (right to left)
for i in range(points_per_side):
    t = i / points_per_side
    x = center_x + square_size/2 - t * square_size
    y = center_y
    z = center_z + square_size/2
    square_path.append([x, y, z])

# Left edge (top to bottom)
for i in range(points_per_side):
    t = i / points_per_side
    x = center_x - square_size/2
    y = center_y
    z = center_z + square_size/2 - t * square_size
    square_path.append([x, y, z])

square_path = np.array(square_path, dtype=np.float32)

print(f"Generated square path with {len(square_path)} waypoints")
print(f"Square size: {square_size} m")
print(f"Center: ({center_x:.3f}, {center_y:.3f}, {center_z:.3f})")


## Setup IK Solver

Configure IK objectives for position, rotation, and joint limits.


In [ ]:
# Residual layout for IK problem:
# [0:3]   - position error (3D)
# [3:6]   - rotation error (3D axis-angle)
# [6:...]  - joint limit violations
total_residuals = 6 + model.joint_coord_count

# Position objective - track end-effector position
pos_obj = ik.IKPositionObjective(
    link_index=ee_index,
    link_offset=wp.vec3(0.0, 0.0, 0.0),
    target_positions=wp.array([ee_pos], dtype=wp.vec3),
    n_problems=1,
    total_residuals=total_residuals,
    residual_offset=0,
)

# Rotation objective - maintain end-effector orientation
rot_obj = ik.IKRotationObjective(
    link_index=ee_index,
    link_offset_rotation=wp.quat_identity(),
    target_rotations=wp.array([wp.vec4(ee_rot[0], ee_rot[1], ee_rot[2], ee_rot[3])], dtype=wp.vec4),
    n_problems=1,
    total_residuals=total_residuals,
    residual_offset=3,
)

# Joint limit objective - keep joints within bounds
joint_limit_obj = ik.IKJointLimitObjective(
    joint_limit_lower=model.joint_limit_lower,
    joint_limit_upper=model.joint_limit_upper,
    n_problems=1,
    total_residuals=total_residuals,
    residual_offset=6,
    weight=10.0,
)

print("IK objectives created")
print(f"  Position objective: residuals [0:3]")
print(f"  Rotation objective: residuals [3:6]")
print(f"  Joint limits: residuals [6:{6 + model.joint_coord_count}]")


In [ ]:
# Joint configuration array (what the solver optimizes)
joint_q = wp.array(model.joint_q, shape=(1, model.joint_coord_count))

# Create IK solver
ik_solver = ik.IKSolver(
    model=model,
    joint_q=joint_q,
    objectives=[pos_obj, rot_obj, joint_limit_obj],
    lambda_initial=0.1,
    jacobian_mode=ik.IKJacobianMode.ANALYTIC,
)

print("IK solver initialized")
print(f"  Lambda (damping): 0.1")
print(f"  Jacobian mode: Analytic")


## Solve IK for Square Trajectory

Run IK solver for each waypoint and visualize the results.


In [ ]:
# Simulation parameters
fps = 60
frame_dt = 1.0 / fps
ik_iterations = 24

# Create viewer
viewer = newton.viewer.ViewerRerun(keep_historical_data=True)
viewer.set_model(model)

# Storage for trajectory trace
trajectory_points = []

print(f"Starting IK simulation at {fps} Hz")
print(f"IK iterations per frame: {ik_iterations}")


In [ ]:
import rerun as rr

# Main simulation loop
sim_time = 0.0

for frame in trange(len(square_path), desc="Solving IK"):
    # Get target position for this frame
    target_pos = square_path[frame]
    
    # Update IK objective with new target
    pos_obj.set_target_position(0, wp.vec3(target_pos[0], target_pos[1], target_pos[2]))
    
    # Solve IK
    ik_solver.solve(iterations=ik_iterations)

    state.joint_q = joint_q[0]
    
    # Evaluate forward kinematics to get body poses
    newton.eval_fk(model, state.joint_q, state.joint_qd, state)
    
    # Get current end-effector position
    body_q_np = state.body_q.numpy()
    ee_current = wp.transform(*body_q_np[ee_index])
    ee_current_pos = wp.transform_get_translation(ee_current)
    trajectory_points.append([ee_current_pos[0], ee_current_pos[1], ee_current_pos[2]])
    
    # Log to viewer
    viewer.begin_frame(sim_time)
    viewer.log_state(state)
    
    # Log target position as red sphere
    rr.log(
        "target",
        rr.Points3D(
            positions=[target_pos],
            radii=0.02,
            colors=[[255, 0, 0]],
        ),
    )
    
    # Log trajectory trace as line
    if len(trajectory_points) > 1:
        rr.log(
            "trajectory",
            rr.LineStrips3D(
                strips=[trajectory_points],
                colors=[[0, 255, 255]],
                radii=0.003,
            ),
        )
    
    viewer.end_frame()
    sim_time += frame_dt

print(f"\nSimulation complete! Total time: {sim_time:.2f} seconds")
print(f"Traced {len(trajectory_points)} points")
viewer


## Analyze IK Performance

Let's check how accurately the end-effector followed the target path.


In [ ]:
# Compute tracking error
trajectory_points = np.array(trajectory_points)
errors = np.linalg.norm(trajectory_points - square_path, axis=1)

print("IK Tracking Performance:")
print(f"  Mean error: {np.mean(errors)*1000:.2f} mm")
print(f"  Max error: {np.max(errors)*1000:.2f} mm")
print(f"  Min error: {np.min(errors)*1000:.2f} mm")
print(f"  Std dev: {np.std(errors)*1000:.2f} mm")


## Summary

In this notebook, we demonstrated:

1. **Loaded Franka FR3 robot** from URDF with fixed base
2. **Created IK objectives** for position, rotation, and joint limits
3. **Generated square trajectory** with 200 waypoints
4. **Solved IK in real-time** at 60 Hz with 24 iterations per frame
5. **Visualized results** with:
   - Red sphere showing target position
   - Cyan line tracing actual end-effector path
   - Full robot visualization

### Key Concepts

**IK Objectives:**
- `IKPositionObjective`: Minimizes position error between end-effector and target
- `IKRotationObjective`: Maintains desired orientation
- `IKJointLimitObjective`: Prevents joint limit violations

**Levenberg-Marquardt Algorithm:**
- Damped least-squares optimization
- Lambda parameter controls damping (stability vs speed)
- Analytic Jacobians for efficiency

**Residuals:**
The IK solver minimizes a weighted sum of residuals:
$$r = [r_{pos}, r_{rot}, r_{limits}]$$

The solver iteratively updates joint angles to minimize:
$$||r||^2 + \lambda ||\Delta q||^2$$

### Next Steps

- Try different trajectories (circle, figure-8, helix)
- Add orientation constraints for specific tasks
- Use multiple end-effectors (dual-arm manipulation)
- Integrate with motion planning for obstacle avoidance
